In [2]:
from transformers import pipeline

import csv
import time
from multiprocessing.pool import ThreadPool
import requests
import os
import PIL.Image as Image
from io import BytesIO 
import numpy as np
from tqdm import tqdm
import shutil
from transformers import AutoTokenizer,GenerationConfig
from transformers import AutoModelForSeq2SeqLM





/root/local/miniconda3/envs/diffuser-gpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
base_file = "./data/eval_images.csv"

In [4]:
tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-12-6")
classifier = pipeline("summarization",min_length=1,max_length=10,do_sample=True)
summarizer = pipeline("summarization", model="sshleifer/distilbart-cnn-12-6")



No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
translation_generation_config = GenerationConfig(
    max_length=10, min_length=1, do_sample=False,num_return_sequences=3,num_beams=3
)
text = "young, curly haired, redhead Natalie Portman  as a optimistic!, cheerful, giddy medieval innkeeper in a dark medieval inn. dark shadows, colorful, candle light,  law contrasts, fantasy concept art by Jakub Rozalski, Jan Matejko, and J.Dickenson"
inputs = tokenizer(text, return_tensors="pt").input_ids
outputs = model.generate(inputs, generation_config=translation_generation_config)
outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
for _ in outputs:
    print(_)
outputs = summarizer(text,max_length=10, min_length=1, do_sample=False,num_return_sequences=3,num_beams=3)
output = []
for _ in outputs:
    output.append({"prompt":_["summary_text"]})
print(output)

</s> Natalie Portman as a optimistic! cheerful,
</s> Natalie Portman as a optimistic, cheerful,
</s> Natalie Portman stars as an optimistic, cheerful
[{'prompt': ' Natalie Portman as a optimistic!'}, {'prompt': ' Natalie Portman as a optimistic,'}, {'prompt': ' Natalie Portman as a cheerful,'}]


In [8]:
# classifier("Paris is the capital and most populous city of France, with an estimated population of 2,175,601 residents as of 2018, in an area of more than 105 square kilometres (41 square miles). The City of Paris is the centre and seat of government of the region and province of Île-de-France, or Paris Region, which has an estimated population of 12,174,880, or about 18 percent of the population of France as of 2017.")
## [{ "summary_text": " Paris is the capital and most populous city of France..." }]

In [ ]:
output = []
with open(base_file, 'r+') as f:
    data = csv.DictReader(f)
    data = [r for r in data]
    pbar = tqdm(total=len(data))
    for row in data:
        prompt = row["prompt"]
        # Coarse-grained text
        # Fine-grained text
        row["fine-grained text"] = prompt
        # row
        outputs = summarizer(text,max_length=10, min_length=1, do_sample=True,num_return_sequences=3,num_beams=3)
        __ = []
        for _ in outputs:
            __.append({"prompt":_["summary_text"]})
        row["coarse-grained prompt"] = __
        # print(output)
        # break
        del row["prompt"]
        output.append(row)
        pbar.update(1)
    pbar.close()

  2%|▏         | 199/8192 [17:13<10:55:29,  4.92s/it]

In [ ]:
with open(f"./data/eval_images_coarse-grained.csv.csv", 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=output[0].keys())
    writer.writeheader()
    writer.writerows(output)